In [3]:
#######################################################################################################################
#                                                                                                                     #
#        The code below contains all the code needed to run the Dash App plot. All that needs to be done is           #
#        download the pokemon.csv dataset file and change the file directory to the folder containing the             #
#        pokemon.csv file                                                                                             #
#                                                                                                                     #
#######################################################################################################################

import os
# set file directory
os.chdir('/Users/david/Desktop/python_datasets')


# data management and manipulation
import pandas as pd
import numpy as np
# matplotlib and seaborn
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
#plotly
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
# Jupyter dash changed to dash
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash.dependencies import Input, Output
from dash import html

# import dataset
file = pd.read_csv('Pokemon.csv')
pokedata = pd.DataFrame(file)

# change NaN to read 'None' because not all pokemon have 2 types
pokedata['Type 2'].fillna('None', inplace=True)
# change '#' column name to 'Number'
pokedata.rename(columns={'#': 'number'}, inplace=True)
#column names: remove spaces, replace '.' with underscore, convert to all lower case
pokedata.columns = pokedata.columns.str.replace(' ', '', regex=False)
pokedata.columns = pokedata.columns.str.replace('.', '_', regex=False)
pokedata.columns = pokedata.columns.str.lower()

# create dash app with multiple chained callbacks to Pokemon Type1

# get unique pokemon type1 and sort alphabetically to input to dropdown options
pokesorted = pokedata.sort_values(['type1', 'name'], ascending=True)
unique_type1 = pokedata['type1'].unique()
unique_type1.sort()

# create dictionary for dash dropdown labels and values
type1_dd_list = [{'label': x, 'value': x} for x in unique_type1]

# create dash app
app = JupyterDash(__name__)

# create app layout
app.layout = html.Div([
    # overall app title
    html.H1('Pokemon Stats'),
    html.Div(
        # divide app into 2 inline-blocks
        children=[
            html.Div(
                # first inline-block
                children=[
                    # type 1 dropdown scatter plot
                    html.H2('Select Pokemon Type'),
                    html.Div(
                        children=[
                            # first dropdown for type1
                            html.H3('First type:'),
                            dcc.Dropdown(id='type1_dd',
                                         options=type1_dd_list,
                                         style={'width': '200px', 'margin': '0 auto'}
                                         )],
                        # format type1 dropdown as inline block with type2 dropdown
                        style={'display': 'inline-block', 'align': 'center'}
                    ),
                    html.Div(
                        children=[
                            # second dropdown for type2
                            html.H3('Optional second type: '),
                            dcc.Dropdown(id='type2_dd',
                                         style={'width': '200px', 'margin': '0 auto'}
                                         )
                        ],
                        # format type2 dropdown as inline block with type1 dropdown
                        style={'display': 'inline-block', 'align': 'center', 'margin': '50px'}
                    ),
                    # scatter plot for type1 and/or type2
                    dcc.Graph(id='type_scatter')
                ],
                # format right side inline block
                style={'display': 'inline-block', 'width': '700px', 'height': '500px', 'text-align': 'center'}
            ),
            html.Div(
                # second inline-block
                children=[
                    # top plot as a histogram
                    html.H2('Histogram of Hit Points (hp)'),
                    dcc.Graph(id='histogram'),
                    # bottom plot as pie chart
                    html.H2('Percentage Legendary'),
                    dcc.Graph(id='pie_chart')
                ],
                # format left inline block
                style={'display': 'inline-block', 'width': '700px', 'height': '10px',
                       'vertical-align': 'top', 'text-align': 'center'}
            ),
        ]
    )
],
    # format overall layout of app
    style={'text-align': 'center'})


# create callback to update type2 dropdown from 'type1_dd'
@app.callback(
    # options is used for output of the chained dropdown for type2
    Output(component_id='type2_dd', component_property='options'),
    Input(component_id='type1_dd', component_property='value')
)
# function to fiter and update scatter plot from type1_dd
def update_type2_dd(type2_dd):
    poke = pokesorted.copy(deep=True)
    # find combinations of type1 and type2
    type1_type2 = poke[['type1', 'type2']].drop_duplicates()
    relevant_type2_dd_options = type1_type2[type1_type2['type1'] == type2_dd]['type2'].values.tolist()

    # create type2_dd dictionary of labels and values
    formatted_relevant_type2_dd_options = [{'label': x, 'value': x} for x in relevant_type2_dd_options]
    return formatted_relevant_type2_dd_options


# create callback for scatterpolot to update with type1 and filter when selecting type2
# all 3 plots will update when type1 is selected and will update again when type2 is selected
@app.callback(
    Output(component_id='type_scatter', component_property='figure'),
    Output(component_id='histogram', component_property='figure'),
    Output(component_id='pie_chart', component_property='figure'),
    Input(component_id='type1_dd', component_property='value'),
    Input(component_id='type2_dd', component_property='value')
)
def update_scatter_type2(type1_dd, type2_dd):
    type2_title = 'All'
    poke = pokesorted.copy(deep=True)

    if type1_dd:
        type1_title = type1_dd
        poke = poke[poke['type1'] == type1_dd]

    # if type2 is not selected do not filter the dataset further
    if type2_dd:
        type2_title = type2_dd
        poke = poke[poke['type2'] == type2_dd]
    # scatter plot
    type_scatter_plot = px.scatter(data_frame=poke, x='attack', y='defense',
                                   custom_data=['name'],
                                   hover_data=['name'],
                                   color='legendary')
    type_scatter_plot.update_xaxes(range=[0, 200])
    type_scatter_plot.update_yaxes(range=[0, 250])
    # histogram
    hp_histogram = px.histogram(poke, x='hp', nbins=20)
    hp_histogram.update_xaxes(range=[0, 250])
    # pie chart
    # for legendary variable: get value counts, convert to dataframe and rename index
    legendary_counts = pd.DataFrame(poke['legendary'].value_counts()).rename(index={False: 'Not Legendary',
                                                                                    True: 'Legendary'})
    legendary_pie = px.pie(legendary_counts, values='legendary', names=legendary_counts.index)
    return type_scatter_plot, hp_histogram, legendary_pie


if __name__ == '__main__':
    app.run_server(mode='external')
#use app.run_server(mode='inline') for the dash app to run within Jupyter notebook

C:\Users\david\Desktop\python_datasets
Dash app running on http://127.0.0.1:8050/
